### Take film links 

<!-- # 8.9 
https://www.momo.vn/cinema/dia-dao-mat-troi-trong-bong-toi-24252/review
# 2.6 
https://www.momo.vn/cinema/black-cab-24264
# 8.1
https://www.momo.vn/cinema/panor-24270/review
# 9.3
https://www.momo.vn/cinema/nu-hon-bac-ty-24185/review
# 7.1
https://www.momo.vn/cinema/anak-kunti-24287/review -->

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
import time

driver = webdriver.Chrome()
driver.get("https://momo.vn/cinema/review")
time.sleep(5)

# Bấm nút "Xem tiếp nhé!" nhiều lần để load thêm phim
for i in range(20):  # Điều chỉnh số lần tùy ý
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        btn = driver.find_element(By.XPATH, "//button[contains(text(),'Xem tiếp nhé')]")
        driver.execute_script("arguments[0].click();", btn)
        print(f"Bấm 'Xem tiếp nhé!' lần {i+1}")
        time.sleep(1.5)
    except NoSuchElementException:
        print("Không còn nút 'Xem tiếp nhé!', dừng lại.")
        break
    except ElementClickInterceptedException:
        print("Lỗi khi click, thử lại sau.")
        time.sleep(2)

# Lấy tất cả link phim
cards = driver.find_elements(By.CSS_SELECTOR, "a[href*='/review']")
film_links = set()

for card in cards:
    href = card.get_attribute("href")
    if href and '/cinema/' in href and '/review' in href:
        film_links.add(href)

film_links = list(film_links)
driver.quit()

# In kết quả
print(f"Số phim lấy được: {len(film_links)}")
for link in film_links:
    print(link)


Bấm 'Xem tiếp nhé!' lần 1
Bấm 'Xem tiếp nhé!' lần 2
Bấm 'Xem tiếp nhé!' lần 3
Bấm 'Xem tiếp nhé!' lần 4
Bấm 'Xem tiếp nhé!' lần 5
Bấm 'Xem tiếp nhé!' lần 6
Bấm 'Xem tiếp nhé!' lần 7
Bấm 'Xem tiếp nhé!' lần 8
Bấm 'Xem tiếp nhé!' lần 9
Bấm 'Xem tiếp nhé!' lần 10
Bấm 'Xem tiếp nhé!' lần 11
Bấm 'Xem tiếp nhé!' lần 12
Bấm 'Xem tiếp nhé!' lần 13
Bấm 'Xem tiếp nhé!' lần 14
Bấm 'Xem tiếp nhé!' lần 15
Bấm 'Xem tiếp nhé!' lần 16
Bấm 'Xem tiếp nhé!' lần 17
Bấm 'Xem tiếp nhé!' lần 18
Bấm 'Xem tiếp nhé!' lần 19
Bấm 'Xem tiếp nhé!' lần 20
Số phim lấy được: 68
https://www.momo.vn/cinema/love-reset-23682/review
https://www.momo.vn/cinema/kayara-24282/review
https://www.momo.vn/cinema/how-to-make-millions-before-grandma-dies-23807/review
https://www.momo.vn/cinema/a-minecraft-movie-24267/review
https://www.momo.vn/cinema/%E1%84%85%E1%85%A9%E1%84%87%E1%85%B5-24295/review
https://www.momo.vn/cinema/yeu-nham-ban-than-24133/review
https://www.momo.vn/cinema/seishun-18-2-kimi-e-to-tsuzuku-michi-23791/review

In [5]:
import re

# Giả sử film_links đã được thu thập ở trước đó
# Ví dụ:
# film_links = [...]

# Định dạng link phim: /cinema/ten-phim-số-id/review
pattern = r"^https://www\.momo\.vn/cinema/[a-z0-9\-]+-\d+/review$"

# Lọc các link phim hợp lệ
valid_film_links = [link for link in film_links if re.match(pattern, link)]

# Ghi vào file
with open("film_links.txt", "w", encoding="utf-8") as f:
    for link in valid_film_links:
        f.write(link + "\n")

# In ra số lượng link hợp lệ
print(f"Đã lọc và lưu {len(valid_film_links)} link phim hợp lệ.")


Đã lọc và lưu 65 link phim hợp lệ.


### Scrape 

In [8]:
# Đọc lại link phim
with open("film_links.txt", "r", encoding="utf-8") as f:
    film_links = [line.strip() for line in f if line.strip()]


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Danh sách 5 phim cần lấy review
# movie_links = [
#     "https://www.momo.vn/cinema/dia-dao-mat-troi-trong-bong-toi-24252/review",
#     "https://www.momo.vn/cinema/black-cab-24264",
#     "https://www.momo.vn/cinema/panor-24270/review",
#     "https://www.momo.vn/cinema/nu-hon-bac-ty-24185/review",
#     "https://www.momo.vn/cinema/anak-kunti-24287/review"
# ]

# Đọc link phim đã lưu từ file
with open("film_links.txt", "r", encoding="utf-8") as f:
    movie_links = [line.strip() for line in f if line.strip()]

# Cấu hình trình duyệt
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
driver.set_page_load_timeout(120)
wait = WebDriverWait(driver, 20)

data = []

for link in movie_links:
    try:
        driver.get(link)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))
        time.sleep(2)

        try:
            movie_title = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
        except:
            movie_title = "Không rõ"

        # Tự động cuộn và nhấn "Xem tiếp nhé!" nhiều lần cho đến khi hết
        # max_scroll = 100  # giới hạn số lần cuộn để tránh vòng lặp vô hạn
        # for i in range(max_scroll):
        #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #     time.sleep(2)
        #     try:
        #         load_more = driver.find_element(By.CSS_SELECTOR, "button.rounded-full.border.border-pink-600")
        #         driver.execute_script("arguments[0].click();", load_more)
        #         print(f"Đã bấm 'Xem tiếp nhé!' lần {i+1}")
        #         time.sleep(2)
        #     except:
        #         print("Không còn nút 'Xem tiếp nhé!', chuyển sang lấy review.")
        #         break
        # Cuộn và bấm nút "Xem tiếp nhé!" tối đa 10 lần nếu có
        for _ in range(10):
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.5)
                btn = driver.find_element(By.XPATH, "//button[contains(text(),'Xem tiếp nhé')]")
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(2)
            except NoSuchElementException:
                break

        # Mở tất cả nội dung ẩn (nút "Xem thêm")
        read_more_buttons = driver.find_elements(By.CSS_SELECTOR, "span.read-or-hide")
        for btn in read_more_buttons:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(0.3)
            except:
                pass

        reviewers = driver.find_elements(By.CSS_SELECTOR, "div.text-md.text-gray-800")
        texts = driver.find_elements(By.CSS_SELECTOR,
            "div.text-md.whitespace-pre-wrap.break-words.leading-relaxed.text-gray-900")
        ratings_blocks = driver.find_elements(By.CSS_SELECTOR,
            "div.flex.items-center.text-md.-ml-1.mb-0\\.5.font-semibold.text-gray-900")

        for idx in range(len(texts)):
            try:
                reviewer = reviewers[idx].text.strip() if idx < len(reviewers) else ""
                review_text = texts[idx].text.strip()

                rating = ""
                label = ""
                if idx < len(ratings_blocks):
                    spans = ratings_blocks[idx].find_elements(By.TAG_NAME, "span")
                    rating = spans[0].text.strip() if len(spans) > 0 else ""
                    label = spans[1].text.strip() if len(spans) > 1 else ""

                try:
                    tags = driver.find_elements(By.CSS_SELECTOR, "span.text-xs.bg-gray-100")
                    tag_string = ", ".join(tag.text.strip() for tag in tags)
                except:
                    tag_string = ""

                data.append({
                    "Tên phim": movie_title,
                    "Người đánh giá": reviewer,
                    "Điểm": rating,
                    "Nhãn cảm xúc": label,
                    "Nội dung đánh giá": review_text,
                    "Tags cảm xúc": tag_string
                })

                print(f"{movie_title} - {reviewer} ({rating})")

            except Exception as e:
                print(f"Lỗi khi xử lý review #{idx+1}: {e}")
                continue

    except Exception as e:
        print(f"Lỗi khi xử lý phim: {link} - {e}")
        continue

driver.quit()

# Xuất dữ liệu
df = pd.DataFrame(data)
df.to_csv("momo_reviews_scrape.csv", index=False, encoding="utf-8-sig")
print("Xong! Dữ liệu đã lưu vào momo_reviews_scrape.csv")


Review phim Yêu Lại Vợ Ngầu trên MoMo - Trần Hồ Phương Hảo (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Pham Thanh Binh (9/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Nguyễn Tất Thành (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Trần Võ Quỳnh Chi (9/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Nguyễn Kim Anh (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Do Thi Nhung (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Đặng Kim Phụng (7/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Lê Hồ Ngọc Thanh (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Nguyễn Thanh Tùng (8/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Huỳnh Gia Huy (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - NGUYỄN THỊ THU THUỶ (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Phạm Đặng Huỳnh Nhu (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Hà Nhật Anh (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Trần Minh Anh (9/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Trần Trà My (10/10)
Review phim Yêu Lại Vợ Ngầu trên MoMo - Nguyễ